Gaussian Elimination with Partial Pivoting
    Solves A x = b and Comparison with NumPy's built-in solve

In [ ]:
import numpy as np

def gauss_elimination_partial_pivot_verbose(A, b):
    """
    Gaussian Elimination with Partial Pivoting
    Solves A x = b
    """
    # Convert to float for safe division
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)

  #  print("Initial Augmented Matrix [A|b]:")
  #  print(np.hstack((A, b.reshape(-1,1))))
  #  print("-" * 50)

    # Forward elimination with partial pivoting
    for k in range(n - 1):
        # Find pivot row (maximum in column k)
        max_row = np.argmax(np.abs(A[k:n, k])) + k
        if A[max_row, k] == 0:
            raise ValueError("Matrix is singular or nearly singular!")

        # Pivoting
        if max_row != k:
            A[[k, max_row]] = A[[max_row, k]]
            b[[k, max_row]] = b[[max_row, k]]
          # print(f"Pivot: swap row {k} with row {max_row}")
          #  print(np.hstack((A, b.reshape(-1,1))))

        # Elimination
        for i in range(k + 1, n):
            factor = A[i, k] / A[k, k]
            A[i, k:] = A[i, k:] - factor * A[k, k:]
            b[i] = b[i] - factor * b[k]
          # print(f"Eliminate row {i}, factor = {factor:.4f}")
         #   print(np.hstack((A, b.reshape(-1,1))))

      #  print("-" * 50)

    # Back substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        if A[i, i] == 0:
            raise ValueError("Zero pivot encountered!")
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]




    return x


# Example usage
if __name__ == "__main__":
    A = np.array([[3.333, 2.0233, -4.233],
                  [2.122, 3.122, 3.122],
                  [5.1222, -3.522, 1.522]])
    b = np.array([3.22, 15.155, 14.744])

    x = gauss_elimination_partial_pivot_verbose(A, b)

    print("Çözüm vektörü x =", x)



y = np.linalg.solve(A, b)
print("Çözüm vektörü built-in y =", y)
print("fark D =", y-x)



Test on ill-conditioned matrices and computing condition numbers

The condition number grows explosively with 𝑛, explaining the loss of accuracy. Even with partial pivoting, Hilbert systems are numerically unstable for 𝑛>10.

n	---  cond(A)---	rel_err_gauss---	rel_err_numpy

2	-->  1.928e+01---	5.661e-16---	5.661e-16

3	-->  5.241e+02---	8.030e-15---	8.023e-15

4	-->  1.551e+04---	9.578e-14---	4.137e-14

5	-->  4.766e+05---	3.493e-13---	1.683e-12

6	--> 1.495e+07---	1.912e-10---	1.424e-10

7--> 4.754e+08---	9.751e-09---	7.637e-09

8-->	  1.526e+10---	3.585e-08---	6.124e-08

9-->	  4.932e+11---	3.137e-06---	3.875e-06

10-->	1.602e+13---	8.242e-05---	8.670e-05

11-->	5.223e+14---	2.112e-03---	8.383e-04

12 --> 1.752e+16---2.428e-01---  3.249e-01




In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import norm, cond, solve

# ------------------------------------------------------------
# 1. Function to build a Hilbert matrix H_ij = 1 / (i + j - 1)
# ------------------------------------------------------------
def hilbert(n):
    i = np.arange(1, n+1)
    j = i.reshape(-1,1)
    return 1.0 / (i + j - 1.0)

# ------------------------------------------------------------
# 2. Gaussian elimination with partial pivoting
# ------------------------------------------------------------
def gauss_elimination_partial_pivot(A, b):
    """
    Solve Ax = b using Gaussian elimination with partial pivoting.
    Returns: (x, swaps)
    """
    A = A.copy().astype(float)
    b = b.copy().astype(float)
    n = A.shape[0]
    swaps = 0   # track number of row swaps

    # --- Forward elimination phase ---
    for k in range(n-1):
        # Partial pivoting: find the largest pivot in column k below diagonal
        pivot_row = np.argmax(np.abs(A[k:, k])) + k
        if abs(A[pivot_row, k]) < 1e-18:
            raise np.linalg.LinAlgError("Matrix is singular to working precision.")

        # Swap rows in A and b if necessary
        if pivot_row != k:
            A[[k, pivot_row], :] = A[[pivot_row, k], :]
            b[[k, pivot_row]] = b[[pivot_row, k]]
            swaps += 1

        # Eliminate entries below pivot
        for i in range(k+1, n):
            m = A[i, k] / A[k, k]          # multiplier
            A[i, k:] = A[i, k:] - m * A[k, k:]
            b[i] = b[i] - m * b[k]

    # --- Back substitution phase ---
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):
        if abs(A[i,i]) < 1e-18:
            raise np.linalg.LinAlgError("Zero diagonal element during back substitution.")
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i,i]

    return x, swaps

# ------------------------------------------------------------
# 3. Test for Hilbert matrices n = 2..12
# ------------------------------------------------------------
rows = []
for n in range(2, 13):
    A = hilbert(n)
    x_true = np.ones(n)
    b = A.dot(x_true)
    kappa = cond(A)  # condition number

    # Solve using custom Gaussian elimination
    try:
        x_gauss, swaps = gauss_elimination_partial_pivot(A, b)
        rel_err_gauss = norm(x_gauss - x_true) / norm(x_true)
        resid_gauss = norm(A.dot(x_gauss) - b) / norm(b)
    except Exception:
        x_gauss = np.full(n, np.nan)
        rel_err_gauss = resid_gauss = np.nan
        swaps = np.nan

    # Solve using numpy.linalg.solve for comparison
    try:
        x_np = solve(A, b)
        rel_err_np = norm(x_np - x_true) / norm(x_true)
        resid_np = norm(A.dot(x_np) - b) / norm(b)
    except Exception:
        rel_err_np = resid_np = np.nan

    rows.append({
        "n": n,
        "cond(A)": kappa,
        "swaps": swaps,
        "rel_err_gauss": rel_err_gauss,
        "resid_gauss": resid_gauss,
        "rel_err_numpy": rel_err_np,
        "resid_numpy": resid_np
    })

# ------------------------------------------------------------
# 4. Display results
# ------------------------------------------------------------
df = pd.DataFrame(rows)
pd.set_option("display.float_format", "{:.3e}".format)
print(df)

     n   cond(A)  swaps  rel_err_gauss  resid_gauss  rel_err_numpy  \
0    2 1.928e+01      0      5.661e-16    0.000e+00      5.661e-16   
1    3 5.241e+02      1      8.030e-15    1.094e-16      8.023e-15   
2    4 1.551e+04      1      9.578e-14    0.000e+00      4.137e-14   
3    5 4.766e+05      2      3.493e-13    1.659e-16      1.683e-12   
4    6 1.495e+07      4      1.912e-10    8.385e-17      1.424e-10   
5    7 4.754e+08      4      9.751e-09    0.000e+00      7.637e-09   
6    8 1.526e+10      4      3.585e-08    2.677e-17      6.124e-08   
7    9 4.932e+11      6      3.137e-06    7.914e-17      3.875e-06   
8   10 1.602e+13      6      8.242e-05    3.334e-17      8.670e-05   
9   11 5.223e+14      8      2.112e-03    1.095e-16      8.383e-04   
10  12 1.752e+16      9      2.428e-01    9.753e-17      3.249e-01   

    resid_numpy  
0     0.000e+00  
1     0.000e+00  
2     0.000e+00  
3     8.666e-17  
4     1.487e-16  
5     6.470e-17  
6     6.558e-17  
7     1.062e-16

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import norm, cond, solve

# ------------------------------------------------------------
# 1. Function to build a Hilbert matrix H_ij = 1 / (i + j - 1)
# ------------------------------------------------------------
def hilbert(n):
    i = np.arange(1, n+1)
    j = i.reshape(-1,1)
    return 1.0 / (i + j - 1.0)

# ------------------------------------------------------------
# 2. Gaussian elimination with partial pivoting
# ------------------------------------------------------------
def gauss_elimination_partial_pivot(A, b):
    """
    Solve Ax = b using Gaussian elimination with partial pivoting.
    Returns: (x, swaps)
    """
    A = A.copy().astype(float)
    b = b.copy().astype(float)
    n = A.shape[0]
    swaps = 0   # track number of row swaps

    # --- Forward elimination phase ---
    for k in range(n-1):
        # Partial pivoting: find the largest pivot in column k below diagonal
        pivot_row = np.argmax(np.abs(A[k:, k])) + k
        if abs(A[pivot_row, k]) < 1e-18:
            raise np.linalg.LinAlgError("Matrix is singular to working precision.")

        # Swap rows in A and b if necessary
        if pivot_row != k:
            A[[k, pivot_row], :] = A[[pivot_row, k], :]
            b[[k, pivot_row]] = b[[pivot_row, k]]
            swaps += 1

        # Eliminate entries below pivot
        for i in range(k+1, n):
            m = A[i, k] / A[k, k]          # multiplier
            A[i, k:] = A[i, k:] - m * A[k, k:]
            b[i] = b[i] - m * b[k]

    # --- Back substitution phase ---
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):
        if abs(A[i,i]) < 1e-18:
            raise np.linalg.LinAlgError("Zero diagonal element during back substitution.")
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i,i]

    return x, swaps

# ------------------------------------------------------------
# 3. Test for Hilbert matrices n = 2..12
# ------------------------------------------------------------
rows = []
for n in range(2, 13):
    A = hilbert(n)
    x_true = np.ones(n)
    b = A.dot(x_true)
    kappa = cond(A)  # condition number

    # Solve using custom Gaussian elimination
    try:
        x_gauss, swaps = gauss_elimination_partial_pivot(A, b)
        rel_err_gauss = norm(x_gauss - x_true) / norm(x_true)
        resid_gauss = norm(A.dot(x_gauss) - b) / norm(b)
    except Exception:
        x_gauss = np.full(n, np.nan)
        rel_err_gauss = resid_gauss = np.nan
        swaps = np.nan

    # Solve using numpy.linalg.solve for comparison
    try:
        x_np = solve(A, b)
        rel_err_np = norm(x_np - x_true) / norm(x_true)
        resid_np = norm(A.dot(x_np) - b) / norm(b)
    except Exception:
        rel_err_np = resid_np = np.nan

    rows.append({
        "n": n,
        "cond(A)": kappa,
        "swaps": swaps,
        "rel_err_gauss": rel_err_gauss,
        "resid_gauss": resid_gauss,
        "rel_err_numpy": rel_err_np,
        "resid_numpy": resid_np
    })

# ------------------------------------------------------------
# 4. Display results
# ------------------------------------------------------------
df = pd.DataFrame(rows)
pd.set_option("display.float_format", "{:.3e}".format)
print(df)

     n   cond(A)  swaps  rel_err_gauss  resid_gauss  rel_err_numpy  \
0    2 1.928e+01      0      5.661e-16    0.000e+00      5.661e-16   
1    3 5.241e+02      1      8.030e-15    1.094e-16      8.023e-15   
2    4 1.551e+04      1      9.578e-14    0.000e+00      4.137e-14   
3    5 4.766e+05      2      3.493e-13    1.659e-16      1.683e-12   
4    6 1.495e+07      4      1.912e-10    8.385e-17      1.424e-10   
5    7 4.754e+08      4      9.751e-09    0.000e+00      7.637e-09   
6    8 1.526e+10      4      3.585e-08    2.677e-17      6.124e-08   
7    9 4.932e+11      6      3.137e-06    7.914e-17      3.875e-06   
8   10 1.602e+13      6      8.242e-05    3.334e-17      8.670e-05   
9   11 5.223e+14      8      2.112e-03    1.095e-16      8.383e-04   
10  12 1.752e+16      9      2.428e-01    9.753e-17      3.249e-01   

    resid_numpy  
0     0.000e+00  
1     0.000e+00  
2     0.000e+00  
3     8.666e-17  
4     1.487e-16  
5     6.470e-17  
6     6.558e-17  
7     1.062e-16

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import norm, cond, solve

# ------------------------------------------------------------
# 1. Function to build a Hilbert matrix H_ij = 1 / (i + j - 1)
# ------------------------------------------------------------
def hilbert(n):
    i = np.arange(1, n+1)
    j = i.reshape(-1,1)
    return 1.0 / (i + j - 1.0)

# ------------------------------------------------------------
# 2. Gaussian elimination with partial pivoting
# ------------------------------------------------------------
def gauss_elimination_partial_pivot(A, b):
    """
    Solve Ax = b using Gaussian elimination with partial pivoting.
    Returns: (x, swaps)
    """
    A = A.copy().astype(float)
    b = b.copy().astype(float)
    n = A.shape[0]
    swaps = 0   # track number of row swaps

    # --- Forward elimination phase ---
    for k in range(n-1):
        # Partial pivoting: find the largest pivot in column k below diagonal
        pivot_row = np.argmax(np.abs(A[k:, k])) + k
        if abs(A[pivot_row, k]) < 1e-18:
            raise np.linalg.LinAlgError("Matrix is singular to working precision.")

        # Swap rows in A and b if necessary
        if pivot_row != k:
            A[[k, pivot_row], :] = A[[pivot_row, k], :]
            b[[k, pivot_row]] = b[[pivot_row, k]]
            swaps += 1

        # Eliminate entries below pivot
        for i in range(k+1, n):
            m = A[i, k] / A[k, k]          # multiplier
            A[i, k:] = A[i, k:] - m * A[k, k:]
            b[i] = b[i] - m * b[k]

    # --- Back substitution phase ---
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):
        if abs(A[i,i]) < 1e-18:
            raise np.linalg.LinAlgError("Zero diagonal element during back substitution.")
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i,i]

    return x, swaps

# ------------------------------------------------------------
# 3. Test for Hilbert matrices n = 2..12
# ------------------------------------------------------------
rows = []
for n in range(2, 13):
    A = hilbert(n)
    x_true = np.ones(n)
    b = A.dot(x_true)
    kappa = cond(A)  # condition number

    # Solve using custom Gaussian elimination
    try:
        x_gauss, swaps = gauss_elimination_partial_pivot(A, b)
        rel_err_gauss = norm(x_gauss - x_true) / norm(x_true)
        resid_gauss = norm(A.dot(x_gauss) - b) / norm(b)
    except Exception:
        x_gauss = np.full(n, np.nan)
        rel_err_gauss = resid_gauss = np.nan
        swaps = np.nan

    # Solve using numpy.linalg.solve for comparison
    try:
        x_np = solve(A, b)
        rel_err_np = norm(x_np - x_true) / norm(x_true)
        resid_np = norm(A.dot(x_np) - b) / norm(b)
    except Exception:
        rel_err_np = resid_np = np.nan

    rows.append({
        "n": n,
        "cond(A)": kappa,
        "swaps": swaps,
        "rel_err_gauss": rel_err_gauss,
        "resid_gauss": resid_gauss,
        "rel_err_numpy": rel_err_np,
        "resid_numpy": resid_np
    })

# ------------------------------------------------------------
# 4. Display results
# ------------------------------------------------------------
df = pd.DataFrame(rows)
pd.set_option("display.float_format", "{:.3e}".format)
print(df)

     n   cond(A)  swaps  rel_err_gauss  resid_gauss  rel_err_numpy  \
0    2 1.928e+01      0      5.661e-16    0.000e+00      5.661e-16   
1    3 5.241e+02      1      8.030e-15    1.094e-16      8.023e-15   
2    4 1.551e+04      1      9.578e-14    0.000e+00      4.137e-14   
3    5 4.766e+05      2      3.493e-13    1.659e-16      1.683e-12   
4    6 1.495e+07      4      1.912e-10    8.385e-17      1.424e-10   
5    7 4.754e+08      4      9.751e-09    0.000e+00      7.637e-09   
6    8 1.526e+10      4      3.585e-08    2.677e-17      6.124e-08   
7    9 4.932e+11      6      3.137e-06    7.914e-17      3.875e-06   
8   10 1.602e+13      6      8.242e-05    3.334e-17      8.670e-05   
9   11 5.223e+14      8      2.112e-03    1.095e-16      8.383e-04   
10  12 1.752e+16      9      2.428e-01    9.753e-17      3.249e-01   

    resid_numpy  
0     0.000e+00  
1     0.000e+00  
2     0.000e+00  
3     8.666e-17  
4     1.487e-16  
5     6.470e-17  
6     6.558e-17  
7     1.062e-16